In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import hashlib
from datetime import datetime

In [75]:
df = pd.read_csv('../../datas/credit_card_purchases.csv')
df = df.dropna().reset_index(drop=True)

In [76]:
def to_hash(cc_num):
    return int(hashlib.sha256(cc_num.encode()).hexdigest(), 16) % 10**8

df['customer_id'] = df['cc_num'].astype('str').apply(to_hash)

In [77]:
df = df.drop(columns=['Unnamed: 0', 'cc_num', 'first', 'last', 'street', 'lat', 'long', 'merch_lat', 'merch_long', 'merch_zipcode', 'trans_num', 'zip'])

In [78]:
df['trans_date_trans_time'] = df['trans_date_trans_time'].str[:10]
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df['dob'] = pd.to_datetime(df['dob'])
df = df.rename(columns={'trans_date_trans_time': 'trans_date'})

In [79]:
def find_age(dob):
    today = datetime.today()
    age = today.year - dob.year

    if (today.month, today.day) < (dob.month, dob.day):
        age -= 1
    return age

df['age'] = df['dob'].apply(find_age)

In [80]:
df['merchant'] = df['merchant'].apply(lambda x: x.replace('fraud_', ''))

In [81]:
df.sample()

,trans_date,merchant,category,amt,gender,city,state,city_pop,job,dob,unix_time,is_fraud,customer_id,age
527866,2019-09-21,"Lowe, Dietrich and Erdman",kids_pets,3.95,F,Phoenix,AZ,1312922,Contractor,1981-10-24,1348257318,0,15584199,42


In [82]:
df = df.iloc[:, [0, 12, 1, 2, 11, 3, 4, 13, 8, 9, 5, 6, 7, 10]]

In [83]:
df.sample(10)

,trans_date,customer_id,merchant,category,is_fraud,amt,gender,age,job,dob,city,state,city_pop,unix_time
39179,2019-01-28,39975485,"Lockman, West and Runte",grocery_pos,0,78.11,M,29,Biomedical engineer,1994-11-24,Leo,IN,5341,1327744111
1033622,2020-05-24,67953539,Graham and Sons,health_fitness,0,24.50,F,64,Landscape architect,1960-01-13,Cisco,IL,478,1369409539
455509,2019-08-18,92444054,Homenick LLC,personal_care,0,65.96,F,48,Financial trader,1976-04-12,Nobleboro,ME,1643,1345301232
512609,2019-09-14,49677644,Kassulke PLC,shopping_net,0,9.54,F,91,Insurance risk surveyor,1932-11-19,Dubre,KY,341,1347587472
25960,2019-01-19,81940053,"Turner, Ziemann and Lehner",food_dining,0,103.53,F,95,Science writer,1929-05-30,Bay Minette,AL,19090,1326958858
277572,2019-06-05,69733026,Huels-Hahn,gas_transport,0,68.94,M,29,"Editor, magazine features",1995-04-19,Hawthorne,CA,93193,1338868760
650333,2019-11-24,83511324,Pouros-Conroy,shopping_pos,0,9.07,F,94,Chemical engineer,1929-08-23,West Columbia,SC,52126,1353753112
599985,2019-10-28,15584199,"Hills, Hegmann and Schaefer",health_fitness,0,17.83,F,42,Contractor,1981-10-24,Phoenix,AZ,1312922,1351436252
279728,2019-06-06,76016291,"Moen, Reinger and Murphy",grocery_pos,0,149.42,F,69,"Civil engineer, contracting",1955-01-20,Humboldt,AZ,1179,1338977025
359074,2019-07-08,84351464,McKenzie-Huels,food_dining,0,199.14,M,33,Exercise physiologist,1991-01-01,Haines City,FL,33804,1341781993


In [84]:
# drop frauds
df = df[df['is_fraud'] == 0]
df = df.drop(columns=['is_fraud'])
dg_df = df[df['merchant'] == 'Dare-Gibson']

In [85]:
dg_df

,trans_date,customer_id,merchant,category,amt,gender,age,job,dob,city,state,city_pop,unix_time
1511,2019-01-01,481839,Dare-Gibson,health_fitness,6.76,M,47,Operations geologist,1976-09-08,Kensington,MD,19054,1325444166
2649,2019-01-02,32381168,Dare-Gibson,health_fitness,104.44,M,80,Health and safety adviser,1944-07-26,Comfrey,MN,914,1325524607
2820,2019-01-02,13264775,Dare-Gibson,health_fitness,124.12,F,53,"Optician, dispensing",1971-08-05,Center Tuftonboro,NH,1368,1325537962
3740,2019-01-03,14843397,Dare-Gibson,health_fitness,55.79,M,49,"Engineer, automotive",1974-12-28,Southfield,MI,75830,1325617149
4756,2019-01-04,64521758,Dare-Gibson,health_fitness,73.92,M,39,Occupational hygienist,1984-11-06,Cazenovia,WI,1360,1325693202
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094699,2020-06-18,16113322,Dare-Gibson,health_fitness,7.50,M,23,Chartered accountant,2000-08-28,De Lancey,NY,782,1371567613
1095277,2020-06-18,74586907,Dare-Gibson,health_fitness,28.93,M,49,English as a second language teacher,1975-04-30,Shrewsbury,MA,35299,1371592446
1096595,2020-06-19,30347331,Dare-Gibson,health_fitness,11.83,M,25,Clothing/textile technologist,1999-05-31,Tekoa,WA,895,1371655266
1096872,2020-06-19,67599391,Dare-Gibson,health_fitness,9.48,M,24,Careers information officer,2000-06-13,Thida,AR,111,1371664667


In [86]:
db_customers = df[df['customer_id'].isin(dg_df['customer_id'])]
db_customers

,trans_date,customer_id,merchant,category,amt,gender,age,job,dob,city,state,city_pop,unix_time
0,2019-01-01,67709251,"Rippin, Kub and Mann",misc_net,4.97,F,36,"Psychologist, counselling",1988-03-09,Moravian Falls,NC,3495,1325376018
1,2019-01-01,29260595,Lind-Buckridge,entertainment,220.11,M,62,Nature conservation officer,1962-01-19,Malad City,ID,4154,1325376051
2,2019-01-01,50636118,Keeling-Crist,misc_pos,41.96,M,38,Dance movement psychotherapist,1986-03-28,Doe Hill,VA,99,1325376186
4,2019-01-01,34306959,Corwin-Collins,gas_transport,71.65,M,76,"Designer, multimedia",1947-08-21,Edinburg,VA,6018,1325376308
5,2019-01-01,64871407,Herzog Ltd,misc_pos,4.27,F,83,Public affairs consultant,1941-03-07,Manor,PA,1472,1325376318
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1100697,2020-06-21,14917032,Stiedemann Inc,misc_pos,37.38,F,43,"Nurse, children's",1980-09-15,North Loup,NE,509,1371816696
1100698,2020-06-21,78379954,Abernathy and Sons,food_dining,51.70,M,44,"Production assistant, television",1979-12-11,Tuscarora,MD,100,1371816739
1100699,2020-06-21,36055952,Stiedemann Ltd,food_dining,105.93,M,56,Naval architect,1967-08-30,High Rolls Mountain Park,NM,899,1371816752
1100700,2020-06-21,66905321,"Reinger, Weissnat and Strosin",food_dining,74.90,M,43,Volunteer coordinator,1980-08-18,Manderson,SD,1126,1371816816


In [93]:
dg_df.groupby('customer_id').agg(
    total_spent = ('amt', 'sum'),
    avg_spent = ('amt', 'mean'),
    num_transactions = ('amt', 'count'),
    avg_age = ('age', 'mean')
    
).sort_values('num_transactions', ascending=False)

,total_spent,avg_spent,num_transactions,avg_age
customer_id,,,,
78933021,555.86,69.482500,8,82.0
35936978,444.52,55.565000,8,53.0
59896227,364.45,52.064286,7,51.0
53730702,529.07,75.581429,7,52.0
37790852,610.39,87.198571,7,52.0
...,...,...,...,...
56585000,22.76,22.760000,1,40.0
56578373,59.38,59.380000,1,31.0
16709634,3.04,3.040000,1,90.0


In [88]:
db_customers[db_customers['customer_id'] == 78933021]

,trans_date,customer_id,merchant,category,amt,gender,age,job,dob,city,state,city_pop,unix_time
34,2019-01-01,78933021,Koepp-Witting,grocery_pos,130.40,F,82,Applications developer,1942-01-06,North Prairie,WI,2328,1325377935
188,2019-01-01,78933021,Kerluke Inc,misc_net,193.90,F,82,Applications developer,1942-01-06,North Prairie,WI,2328,1325386479
645,2019-01-01,78933021,McGlynn-Heathcote,misc_net,1.70,F,82,Applications developer,1942-01-06,North Prairie,WI,2328,1325411576
788,2019-01-01,78933021,"Greenfelder, Bartoletti and Davis",misc_net,94.67,F,82,Applications developer,1942-01-06,North Prairie,WI,2328,1325418698
1134,2019-01-01,78933021,Kilback LLC,food_dining,22.83,F,82,Applications developer,1942-01-06,North Prairie,WI,2328,1325431528
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098696,2020-06-20,78933021,Kertzmann LLC,health_fitness,23.19,F,82,Applications developer,1942-01-06,North Prairie,WI,2328,1371744263
1099248,2020-06-20,78933021,"Hahn, Douglas and Schowalter",travel,6.11,F,82,Applications developer,1942-01-06,North Prairie,WI,2328,1371764153
1099484,2020-06-20,78933021,Dare-Marvin,entertainment,32.10,F,82,Applications developer,1942-01-06,North Prairie,WI,2328,1371771501
1100165,2020-06-21,78933021,Kerluke Inc,misc_net,35.68,F,82,Applications developer,1942-01-06,North Prairie,WI,2328,1371797348


In [ ]:
db_customers[db_customers['customer_id'] == 78933021].value_counts('category')

In [523]:
# sns.kdeplot(x=df['age'])

In [524]:
# sns.kdeplot(x=df['amt'])

In [517]:
# sns.boxplot(x='age', y='state', data=df)

In [518]:
# sns.boxplot(x='amt', y='state', data=df)

In [519]:
# def remove_outliers(group):
#     Q1_age = group['age'].quantile(0.25)
#     Q3_age = group['age'].quantile(0.75)
#     IQR_age = Q3_age - Q1_age

#     lower_bound_age = Q1_age - 1.5 * IQR_age
#     upper_bound_age = Q3_age + 1.5 * IQR_age

#     Q1_amt = group['amt'].quantile(0.25)
#     Q3_amt = group['amt'].quantile(0.75)
#     IQR_amt = Q3_amt - Q1_amt

#     lower_bound_amt = Q1_amt - 1.5 * IQR_amt
#     upper_bound_amt = Q3_amt + 1.5 * IQR_amt

#     return group[(group['age'] >= lower_bound_amt) & (group['age'] <= upper_bound_amt) &
#                  (group['amt'] >= lower_bound_age) & (group['amt'] <= upper_bound_age)]

In [520]:
# df = df.groupby('state').apply(remove_outliers).reset_index(drop=True)

In [521]:
# sns.boxplot(x='age', y='state', data=df)

In [522]:
# sns.boxplot(data=df, x='amt', y='state')